In [1]:
# MRI VISUALIZATION from OASIS-3
### Jack Le - Polygence

In [2]:
import nibabel as nib
import ipywidgets as widgets
from scipy import ndimage
import os, random
import matplotlib.gridspec as gridspec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

df = pd.read_csv('OASIS/oasis_3.csv')
df

,Subject,MR ID,id,Age,M/F,dx1,mmse,cdr,apoe,TOTAL_HIPPOCAMPUS_VOLUME,...,rhCortexVol,CortexVol,SubCortGrayVol,TotalGrayVol,SupraTentorialVol,lhCorticalWhiteMatterVol,rhCorticalWhiteMatterVol,CorticalWhiteMatterVol,L.SurfArea,R.SurfArea
0,OAS30001,OAS30001_MR_d3132,OAS30001_Freesurfer53_d3132,73.0,F,Cognitively normal,30.0,0.0,23.0,6861.9,...,178031.558882,359975.257636,48400.0,491102.257636,7.736716e+05,174372.329393,173244.012238,347616.341631,67598.1,67185.8
1,OAS30001,OAS30001_MR_d0129,OAS30001_Freesurfer53_d0129,65.0,F,Cognitively normal,30.0,0.0,23.0,7678.9,...,187528.786036,379446.180091,50687.0,517683.180091,8.105851e+05,184600.488060,182662.445419,367262.933479,70168.1,69483.8
2,OAS30001,OAS30001_MR_d2430,OAS30001_Freesurfer53_d2430,71.0,F,Cognitively normal,30.0,0.0,23.0,7105.9,...,178872.680224,357784.489639,49058.0,487405.489639,7.779313e+05,175955.968883,178172.812666,354128.781549,67905.7,68000.2
3,OAS30001,OAS30001_MR_d0757,OAS30001_Freesurfer53_d0757,67.0,F,Cognitively normal,29.0,0.0,23.0,7648.2,...,177566.874682,362040.150904,50071.0,500699.150904,7.993419e+05,185224.779932,188151.990316,373376.770247,69142.3,68558.8
4,OAS30002,OAS30002_MR_d2345,OAS30002_Freesurfer53_d2345,73.0,M,Cognitively normal,29.0,0.0,34.0,7833.2,...,230240.532783,457342.035802,56773.0,607473.035802,1.051714e+06,239168.338419,245361.377267,484529.715686,83138.1,85742.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,OAS31169,OAS31169_MR_d0620,OAS31169_Freesurfer53_d0620,64.0,F,Cognitively normal,29.0,0.0,34.0,6828.5,...,227573.947792,450474.518776,49085.0,593751.518776,9.897836e+05,208218.439241,211361.618639,419580.057880,79639.5,80013.5
2164,OAS31170,OAS31170_MR_d2410,OAS31170_Freesurfer53_d2410,71.0,F,AD dem distrubed social- with,18.0,1.0,34.0,4501.3,...,196406.732825,383465.515848,45633.0,506151.515848,8.747602e+05,206082.644662,214673.053871,420755.698533,76344.0,79045.4
2165,OAS31171,OAS31171_MR_d0752,NaN,68.0,F,AD dem distrubed social- with,21.0,1.0,44.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2166,OAS31172,OAS31172_MR_d1717,OAS31172_Freesurfer53_d1717,75.0,F,Cognitively normal,29.0,0.0,23.0,7464.3,...,204278.557883,408581.059329,49411.0,534192.059329,9.296810e+05,210892.330905,210087.600539,420979.931444,81945.9,80006.1


## Data Cleaning

In [3]:
df = df.dropna(axis=1, how='all') # Drop any empty columns
df = df.dropna(axis=0, how='any') # Drop any rows with empty values
df = df.rename(columns={'id':'Freesurfer ID', 'dx1':'Diagnosis', # Rename columns
                        'TOTAL_HIPPOCAMPUS_VOLUME':'TotalHippocampusVol'})
df = df.sort_values(by=['Subject', 'Age']) # Sort the columns by subject, then age
df = df.drop_duplicates(subset='Subject', keep='first') # Keep only the first visit; this is possible because
                                                        # df is sorted by age
df = df.reset_index(drop=True) # Reset the index

In [4]:
def label_disease (row):
    if row['cdr'] < 0.5:
        return 0
    elif row['Diagnosis'] == 'Cognitively normal':
        return 0
    else:
        return 1
# Labels the diagnosis numerically
df.insert(6, 'Demented', df.apply (lambda row : label_disease(row), axis=1))
df

,Subject,MR ID,Freesurfer ID,Age,M/F,Diagnosis,Demented,mmse,cdr,apoe,...,rhCortexVol,CortexVol,SubCortGrayVol,TotalGrayVol,SupraTentorialVol,lhCorticalWhiteMatterVol,rhCorticalWhiteMatterVol,CorticalWhiteMatterVol,L.SurfArea,R.SurfArea
0,OAS30001,OAS30001_MR_d0129,OAS30001_Freesurfer53_d0129,65.0,F,Cognitively normal,0,30.0,0.0,23.0,...,187528.786036,379446.180091,50687.000000,517683.180091,8.105851e+05,184600.488060,182662.445419,367262.933479,70168.1,69483.8
1,OAS30002,OAS30002_MR_d0653,OAS30002_Freesurfer53_d0653,68.0,M,Cognitively normal,0,30.0,0.0,34.0,...,236894.539679,471180.950626,60386.000000,628934.950626,1.085865e+06,253411.175744,257796.379281,511207.555026,85010.3,87126.4
2,OAS30003,OAS30003_MR_d0558,OAS30003_Freesurfer53_d0558,60.0,F,Cognitively normal,0,30.0,0.0,33.0,...,214696.395609,421337.406916,59540.000000,569622.406916,9.538035e+05,226794.049522,229032.005946,455826.055467,77748.9,80033.3
3,OAS30004,OAS30004_MR_d1101,OAS30004_Freesurfer53_d1101,58.0,F,Cognitively normal,0,30.0,0.0,23.0,...,216737.873611,436061.628148,54136.000000,585563.628148,9.794057e+05,238398.089623,234433.968088,472832.057710,86003.6,84289.7
4,OAS30005,OAS30005_MR_d0143,OAS30005_Freesurfer51_d0143,48.0,F,Cognitively normal,0,29.0,0.0,33.0,...,234532.952497,472022.810547,198617.449249,670640.259796,1.019123e+06,255716.750000,253689.734375,509406.484375,82802.7,81709.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,OAS31167,OAS31167_MR_d0064,OAS31167_Freesurfer50_d0064,51.0,M,Cognitively normal,0,30.0,0.0,33.0,...,234561.404619,468791.982536,175293.167969,644085.150505,1.008741e+06,252444.625000,255790.125000,508234.750000,86945.6,87200.2
1018,OAS31168,OAS31168_MR_d0148,OAS31168_Freesurfer53_d0148,65.0,M,Cognitively normal,0,30.0,0.0,44.0,...,235478.553430,468562.523147,57431.000000,617912.523147,1.012598e+06,229996.472108,227638.300745,457634.772852,86355.3,85926.8
1019,OAS31169,OAS31169_MR_d0620,OAS31169_Freesurfer53_d0620,64.0,F,Cognitively normal,0,29.0,0.0,34.0,...,227573.947792,450474.518776,49085.000000,593751.518776,9.897836e+05,208218.439241,211361.618639,419580.057880,79639.5,80013.5
1020,OAS31170,OAS31170_MR_d2410,OAS31170_Freesurfer53_d2410,71.0,F,AD dem distrubed social- with,1,18.0,1.0,34.0,...,196406.732825,383465.515848,45633.000000,506151.515848,8.747602e+05,206082.644662,214673.053871,420755.698533,76344.0,79045.4


In [5]:
# Creates a sample from df, with equal distribution of CN and Demented
samp_df = df.groupby('Demented').apply(lambda x: x.sample(n=10, random_state=1)).reset_index(drop = True)
# the next two lines are commented out to prevent the mri_ids csv from getting messed up
# mri_ids = samp_df['MR ID']
# mri_ids.to_csv('OASIS/mri_id.csv')
samp_df.head()

,Subject,MR ID,Freesurfer ID,Age,M/F,Diagnosis,Demented,mmse,cdr,apoe,...,rhCortexVol,CortexVol,SubCortGrayVol,TotalGrayVol,SupraTentorialVol,lhCorticalWhiteMatterVol,rhCorticalWhiteMatterVol,CorticalWhiteMatterVol,L.SurfArea,R.SurfArea
0,OAS30319,OAS30319_MR_d0043,OAS30319_Freesurfer53_d0043,52.0,F,Cognitively normal,0,28.0,0.0,33.0,...,215463.137759,432202.575736,59682.0,600447.575736,9.684294e+05,231420.662820,227742.160614,459162.823434,79301.4,78742.4
1,OAS30187,OAS30187_MR_d0070,OAS30187_Freesurfer53_d0070,69.0,M,Cognitively normal,0,30.0,0.0,22.0,...,206481.344712,409048.927762,49082.0,540514.927762,9.156372e+05,212237.003726,213698.315772,425935.319498,80261.6,80440.1
2,OAS30707,OAS30707_MR_d0453,OAS30707_Freesurfer53_d0453,84.0,M,Cognitively normal,0,29.0,0.0,34.0,...,209777.844623,417006.254842,53288.0,557491.254842,9.979297e+05,212839.014461,218128.475910,430967.490370,87176.2,86303.2
3,OAS30713,OAS30713_MR_d0095,OAS30713_Freesurfer53_d0095,70.0,M,Cognitively normal,0,30.0,0.0,33.0,...,241160.461674,481131.578644,60019.0,632272.578644,1.118171e+06,273108.315782,273504.602185,546612.917967,92350.2,92192.2
4,OAS31107,OAS31107_MR_d0912,OAS31107_Freesurfer53_d0912,69.0,F,Cognitively normal,0,30.0,0.0,33.0,...,242831.843654,487340.610810,55289.0,658025.610810,1.082691e+06,251933.107555,248947.377523,500880.485078,85370.6,85872.0


In [43]:
def choose_image(type):
    # Chooses a random image from the chosen group
    dir = "OASIS/MRI_DATA/DEM" if type == 'Demented' else "OASIS/MRI_DATA/CN"
    id_path = random.choice(os.listdir(dir))
    mri_dir = os.path.join(dir, id_path)
    mri_path = random.choice(os.listdir(mri_dir))
    img_dir = os.path.join(mri_dir, mri_path)
    img_path = [file for file in os.listdir(img_dir) if file.endswith('.gz')]
    scan_dir = os.path.join(img_dir, img_path[0])
    print(f'Path of current MRI: {scan_dir}')

    # Loads image
    image_obj = nib.load(scan_dir)
    image_data = image_obj.get_fdata()
    height, width, depth = image_data.shape

    # Sets up data table at bottom
    patient_data = samp_df.loc[samp_df['MR ID'] == id_path].drop(df.columns.difference(['Age', 'MR ID', 'M/F', 'mmse', 'apoe', 
    'TotalHippocampusVol', 'IntraCranialVol', 'TotalGrayVol']), 1)

    return scan_dir, patient_data, image_data, height, width, depth
scan_dir, patient_data, image_data, height, width, depth = choose_image("Cognitively normal")

In [55]:
hi = widgets.IntSlider(value=height/2, max=height-1, continous_updates=False)
wi = widgets.IntSlider(value=width/2, max=width-1, continous_updates=False)
de = widgets.IntSlider(value=depth/2, max=depth-1, continous_updates=False)
co = widgets.ToggleButtons(options=['Gray', 'Color'])
st = widgets.ToggleButtons(options=['Cognitively Normal', 'Demented'])

def tet(change):
    print(change.new)

st.observe(tet, 'value')

top_box = widgets.HBox([widgets.Label("Side Layer"), hi,
                        widgets.Label("Front Layer"), wi,
                        widgets.Label("Top Layer"), de])
bot_box = widgets.HBox([widgets.Label("Color Map"), co,
                        widgets.Label("Patient Type"), st])
ui = widgets.VBox([top_box, bot_box])

test = 'test'

curr_value = st.value
def show_graph(img, height, width, depth, color, type):
    global curr_value, scan_dir, patient_data
    
    print(scan_dir)
    if curr_value != st.value:
        curr_value = st.value
        choose_image(st.value)
    
    # Plots brain scans in all three dimensions
    fig = plt.figure(figsize=(15,7))
    gs = gridspec.GridSpec(nrows=3, ncols=3)

    ax0 = fig.add_subplot(gs[0:2,0])
    ax1 = fig.add_subplot(gs[0:2,1])
    ax2 = fig.add_subplot(gs[0:2,2])

    ax0.set_title("Side View")
    ax0.axis('off')
    ax1.set_title("Front View")
    ax1.axis('off')
    ax2.set_title("Top View")
    ax2.axis('off')

    colormap = 'gray' if color == 'Gray' else 'viridis'
    ax0.imshow(ndimage.rotate(img[height, :, :],90), cmap=colormap)
    ax1.imshow(ndimage.rotate(img[:, width, :],90), cmap=colormap)
    ax2.imshow(ndimage.rotate(img[:, :, depth],90), cmap=colormap)

    # Bottom data table
    bbox=[0, 0, 1, 1]
    fig.suptitle(f'3D Brain Scans for {type} Patient')
    data = fig.add_subplot(gs[2,:])
    data.axis('off')
    tbl = data.table(cellText=patient_data.values, bbox=bbox, colLabels=patient_data.columns)
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(10)
    fig.tight_layout()

#widgets.interact(demented, img=widgets.fixed(image_data), height=h, width=w, depth=d, gray=g)
out = widgets.interactive_output(show_graph, {'img':widgets.fixed(image_data), 'height':hi, 'width':wi, 'depth':de, 'color':co, 'type':st})
out.layout.height = '600px'

display(ui, out)

Output(layout=Layout(height='600px'))

Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
Demented


In [54]:
test = widgets.ToggleButtons(options=['Cognitively Normal', 'Demented'])
def tet(change):
    print(change.new)

test.observe(tet, 'value')
test

ToggleButtons(options=('Cognitively Normal', 'Demented'), value='Cognitively Normal')

Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
Demented
Cognitively Normal
